<a href="https://colab.research.google.com/github/ishgirwan/omdena_hdi/blob/master/training_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Install xla for tpu support
#!curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
#!python pytorch-xla-env-setup.py --version nightly --apt-packages libomp5 libopenblas-dev

In [3]:
!pip install rasterio
#pip install pytorch-lightning
!pip install pytorch-lightning-bolts
#!pip install albumentations
!pip install git+https://github.com/PytorchLightning/pytorch-lightning.git@master --upgrade

  Cloning https://github.com/PytorchLightning/pytorch-lightning.git (to revision master) to /tmp/pip-req-build-pjr31q6j
  Running command git clone -q https://github.com/PytorchLightning/pytorch-lightning.git /tmp/pip-req-build-pjr31q6j
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for pytorch-lightning: filename=pytorch_lightning-1.0.0rc2-cp36-none-any.whl size=469150 sha256=0191cf12e5d7fe897a891a3ac2a4c235b9181733cd57b751ca5234e58e4acd1f
  Stored in directory: /tmp/pip-ephem-wheel-cache-z7_luj8n/wheels/71/30/98/7d39a914d2f5563909745100a78950c34221690e6fe574a915
Successfully built pytorch-lightning
  Found existing installation: pytorch-lightning 0.10.0
    Uninstalling pytorch-lightning-0.10.0:
      Successfully uninstalled pytorch-lightning-0.10.0


In [4]:
import numpy as np
import pandas as pd
import rasterio

import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, sampler
from torchvision import transforms

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pl_bolts.callbacks import PrintTableMetricsCallback
from pytorch_lightning.metrics import MeanAbsoluteError
#import albumentations as A

#from sklearn.preprocessing import MinMaxScaler    
#from sklearn.metrics import r2_score

import glob
import os

# visualisation
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
csv_path = '/content/drive/My Drive/omdena/hdi_with_geometry.csv'
root_dir = '/content/drive/My Drive/Landsat/'

In [6]:
class MyDataset(Dataset):
    """
    Generate normalized, rescaled and transformed datasets
    """

    def __init__(self, dataset_type, transform=None):
        
        super().__init__()
        self.df = dataset_type
        self.transform = transform

    def __len__(self):
        
        return len(self.df)
    
    def __getitem__(self, idx):
       
        if torch.is_tensor(idx):
              idx = idx.tolist()

        # generate image sample
        image_path = self.df['image_path'].iloc[idx] 
        image_sample = rasterio.open(str(image_path), "r")
        bands = [i for i in range(1, image_sample.count+1)]
        image_sample = image_sample.read(bands)
        image_sample = image_sample.astype('float32')

        # generate hdi sample

        hdi_sample = self.df['HDI'].iloc[idx]

        # Normalize the image sample and rescale it between 0 and 1
        for ch in range(image_sample.shape[0]):
            # standardize
            image_sample[ch] = (image_sample[ch] - np.nanmean(image_sample[ch])) / np.nanstd(image_sample[ch])
            
            # normalize
            image_sample[ch] = (image_sample[ch] - np.nanmin(image_sample[ch])) / (np.nanmax(image_sample[ch]) - np.nanmin(image_sample[ch]))
        
        # convet nan to 0
        image_sample[np.isnan(image_sample)] = 0

        if self.transform:
            image_sample = self.transform(image_sample)

        return image_sample.permute(1, 0, 2),  hdi_sample.astype('float32')
    

In [7]:
model = torchvision.models.segmentation.fcn_resnet50() 
#model

In [8]:
model.backbone.conv1 = nn.Conv2d(6, 64, kernel_size=3, stride=1, padding=3, bias=False)
model.classifier[4] = nn.Sequential(nn.Conv2d(512, 1, kernel_size=(1, 1), stride=(1, 1)), nn.Sigmoid())

In [9]:
from pytorch_lightning.callbacks import EarlyStopping

# default used by the Trainer
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=4,
    strict=False,
    verbose=True,
    mode='min')

from pytorch_lightning.callbacks import ModelCheckpoint

# DEFAULTS used by the Trainer
checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/My Drive/omdena/ckpt',
    save_top_k=1,
    verbose=True,
    monitor='val_loss',
    mode='min',
    prefix=''
)


In [17]:
def my_collate(batch):

    max_wh = 0

    for item in batch:
        image = item[0]
        w = image.shape[1]
        h = image.shape[2]
        max_i = np.max([w, h])
        if max_i > max_wh:
            max_wh = max_i
    
    #print(max_wh)

    data = []

    for item in batch:
        image = item[0]
        rows = image.shape[1]
        cols = image.shape[2]
        rows_diff = max_wh - rows
        cols_diff = max_wh - cols
        cols_half = int(cols_diff / 2)
        rows_half = int(rows_diff / 2)
        padding = (cols_half, cols_diff-cols_half, rows_half, rows_diff-rows_half)
        image_pad = F.pad(image, padding, 'constant', 0)
        #print(image.shape)
        #print(image_pad.shape)
        #print(type(image_pad))
        data.append(image_pad)

    target = [item[1] for item in batch]
    target = torch.LongTensor(target)
    #data = torch.Tensor(data)
    return [data, target]

In [32]:
class Model(pl.LightningModule):

    def __init__(self, model, batch_size=1, learning_rate=.001):
        super().__init__()
        self.learning_rate = learning_rate
        self.save_hyperparameters()
        self.model = model
        self.batch_size = batch_size

    def forward(self, x):
        x = self.model(x)
        return torch.mean(x['out'])

    def prepare_data(self):

        df = pd.read_csv(csv_path)
        df['image_path'] = root_dir + df['unique code'].astype(str) + '.tif' 
        df = df.sample(frac=1, random_state=1).reset_index(drop=True)
        # split the dataset
        train, validate, test = np.split(df, [int(.95*len(df)), int(.975*len(df))]) 

        # transforms
        train_transform = transforms.Compose([
                                transforms.ToTensor()
                                ])

        validate_transform = transforms.Compose([
                                transforms.ToTensor()
                                ])
        # create datasets for training, validation and test
        self.train_dataset = MyDataset(dataset_type=train, transform=train_transform)
        self.validate_dataset = MyDataset(dataset_type=validate, transform=validate_transform)
        self.test_dataset = MyDataset(dataset_type=test, transform=validate_transform) 
    
    def train_dataloader(self):
        return DataLoader(self.train_dataset, self.batch_size, shuffle=True, num_workers=1, collate_fn=my_collate, pin_memory=True)

    def val_dataloader(self):
        return DataLoader(self.validate_dataset, self.batch_size, num_workers=1,collate_fn=my_collate, pin_memory=True) 

    def test_dataloader(self):
        return DataLoader(self.test_dataset, self.batch_size, num_workers=1, collate_fn=my_collate, pin_memory=True) 

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=2, verbose=True)
        return {'optimizer': optimizer, 'lr_scheduler': scheduler, 'monitor': 'val_loss'}

    def training_step(self, batch, batch_idx):
        x, y = batch

        #y = torch.stack(y)
        x = torch.stack(x)
        print(x.shape)
        print(y.shape)
        #x = np.array(x)
        #y = np.array(y)
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y.float())
        #print('y_hat:{} y:{}'.format(y_hat, y))
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        x = torch.stack(x)
        #x = np.array(x)
        #print(type(x))
        #print(x.size)
        #y = np.array(y.cpu())
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y.float())
        self.log('val_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        #self.log('val_R-square', r2_score(y, y_hat),  prog_bar=True, on_step=False, on_epoch=True)
        return loss

    #def validation_epoch_end(self, outputs):
    #   avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
    #  return {'val_loss': avg_loss}

    def test_step(self, batch, batch_idx):
        x, y = batch
        x = torch.stack(x)
        #x = np.array(x)
        #y = np.array(y)
        y_hat = self(x)
        loss = F.mse_loss(y_hat, y.float())
        self.log('test_loss', loss, on_step=True, on_epoch=False, prog_bar=True, logger=True)
        #self.log('val_R-square', r2_score(y, y_hat), loss, prog_bar=True, on_step=False, on_epoch=True)
        return loss

    def backward(self, trainer, loss, optimizer, optimizer_idx):
        loss.backward()

    def optimizer_step(self, current_epoch, batch_idx, optimizer, 
      optimizer_idx, second_order_closure=None, 
       on_tpu=False, using_native_amp=False, using_lbfgs=False):
        optimizer.step()


In [33]:
# init model
model_one = Model(model, batch_size=4)
#from pytorch_lightning.core.memory import ModelSummary
#ModelSummary(model_one, mode='full')

In [35]:
#seed
pl.seed_everything(1234)

from pytorch_lightning.loggers import TensorBoardLogger
logger = TensorBoardLogger('/content/drive/My Drive/omdena/tb_logs', name='my_model')

#train
root_path = '/content/drive/My Drive/omdena'
trainer = pl.Trainer(gpus=1, logger=logger, checkpoint_callback=checkpoint_callback, limit_test_batches=.01, limit_train_batches=.001, limit_val_batches=.01, progress_bar_refresh_rate=50, accumulate_grad_batches=1, fast_dev_run=False,\
                    default_root_dir=root_path, auto_lr_find=True, gradient_clip_val=0.5,\
                    profiler=True,  max_epochs=2, callbacks=[early_stop, PrintTableMetricsCallback()])

trainer.fit(model_one)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type | Params
-------------------------------
0 | model | FCN  | 32 M  


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:72: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


torch.Size([4, 6, 209, 209])
torch.Size([4])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:59: UserWarning: Using a target size (torch.Size([4])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


torch.Size([4, 6, 229, 229])
torch.Size([4])
torch.Size([4, 6, 134, 134])
torch.Size([4])
torch.Size([4, 6, 124, 124])
torch.Size([4])


Epoch 0: val_loss reached 0.33533 (best 0.33533), saving model to /content/drive/My Drive/omdena/ckpt/epoch=0-v0.ckpt as top 1
train_loss_step│train_loss│val_loss_epoch│val_loss│train_loss_epoch
───────────────────────────────────────────────────────────────────
0.03538372740149498│0.10232947021722794│0.335330605506897│0.335330605506897│0.10232947021722794


torch.Size([4, 6, 189, 189])
torch.Size([4])
torch.Size([4, 6, 180, 180])
torch.Size([4])
torch.Size([4, 6, 202, 202])
torch.Size([4])
torch.Size([4, 6, 277, 277])
torch.Size([4])


Epoch 1: val_loss reached 0.00353 (best 0.00353), saving model to /content/drive/My Drive/omdena/ckpt/epoch=1.ckpt as top 1
train_loss_step│train_loss│val_loss_epoch│val_loss│train_loss_epoch
───────────────────────────────────────────────────────────────────
0.03538372740149498│0.10232947021722794│0.335330605506897│0.335330605506897│0.10232947021722794
0.016812579706311226│0.021152235567569733│0.00352984550409019│0.00352984550409019│0.021152235567569733


Profiler Report

Action              	|  Mean duration (s)	|  Total time (s) 
-----------------------------------------------------------------
on_fit_start        	|  3.0708e-05     	|  3.0708e-05     
on_validation_start 	|  0.022841       	|  0.068524       
on_validation_epoch_start	|  4.7614e-05     	|  0.00014284     
on_validation_batch_start	|  0.00015678     	|  0.00047035     
validation_step_end 	|  2.9026e-05     	|  8.7077e-05     
on_validation_batch_end	|  0.00011053     	|  0.00033159     
on_validation_epoch_end	|  6

1

In [ ]:
# Start tensorboard.
%reload_ext tensorboard
%tensorboard --logdir='/content/drive/My Drive/omdena/tb_logs'

In [ ]:
# test
trainer.test()

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss': tensor(0.0117, device='cuda:0'),
 'train_loss': tensor(0.0111, device='cuda:0'),
 'val_loss': tensor(0.0122, device='cuda:0')}
--------------------------------------------------------------------------------



[{'test_loss': 0.011672185733914375,
  'train_loss': 0.011075790040194988,
  'val_loss': 0.012162690050899982}]